In [9]:
#!pip install xmltodict

ERROR: Could not find a version that satisfies the requirement StringIO (from versions: none)
ERROR: No matching distribution found for StringIO


In [13]:
import xmltodict
data=xmltodict.parse(open(filepath, encoding="UTF-8").read())

In [15]:
#explore
data['osm'].keys()

odict_keys(['@version', '@generator', 'note', 'meta', 'bounds', 'node', 'way', 'relation'])

In [55]:
#process to get a required nodes
result=[]

for w in data['osm']['way']:    
    if not 'tag' in w:
        continue
    ts=w['tag']    
    if not isinstance(ts, list):
        ts = [ts]
    if not 'highway' in [m['@k'] for m in ts]:
        continue
    nds=w['nd']    
    for i in range(len(nds)-1):
        node1=nds[i]['@ref']
        node2=nds[i+1]['@ref']
        result.append((node1, node2))
        result.append((node2, node1))    

In [59]:
len(result)

26098

In [64]:
node_id_list=set([v for v, _ in result] + [v for _, v in result])
len(node_id_list)

11640

In [68]:
#get required coordinates 
ps=[]
for node in data['osm']['node']:
    node_id=node['@id']
    if node_id in node_id_list:
        ps.append((node_id, float(node['@lat']), float(node['@lon'])))
        
len(ps)    

11640

In [69]:
with open('bruges_locations.csv', 'w') as node_file:
    node_file.write('id,lat,lon\n')
    for id1, lat, lon in ps:
        node_file.write(f'{id1},{lat},{lon}\n')

In [71]:
with open('bruge_road_segments.csv', 'w') as edge_file:
    edge_file.write('src,dst\n')
    for src, dst in result:
        edge_file.write(f'{src},{dst}\n')